In [ ]:
from rdflib import Dataset, URIRef
from rdflib import Namespace
import csv

In [ ]:
CRM = Namespace('http://cidoc-crm.org/cidoc-crm/')

In [ ]:
# the URI for our named graph that will contain our data
# CHANGE URI to the one used previously and uncomment
# anno_named_graph_uri = 'http://example.org/ns/users/robcast/genesis-example'
anno_named_graph_uriref = URIRef(anno_named_graph_uri)

In [ ]:
# open a connection to the Blazegraph triple store
blazegraph_uri = 'http://localhost:8080/blazegraph/namespace/kb/sparql'
store = Dataset('SPARQLUpdateStore')
store.open((blazegraph_uri, blazegraph_uri))

In [ ]:
# find our named graph if it exists or create a new one
anno_named_graph = None
for g in store.graphs():
    if g.identifier == anno_named_graph_uriref:
        anno_named_graph = g

if anno_named_graph is None:
    anno_named_graph = store.graph(anno_named_graph_uriref)

In [ ]:
# read relations from CSV file
with open('recogito-genesis-relations.csv', newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        source_name = row['source']
        target_name = row['target']
        rel_type = row['relation']
        print(f"relation: {source_name} - {rel_type} -> {target_name}")
        
        if rel_type == 'is father of':
            source_person = URIRef(f"{anno_named_graph_uri}/persons/{source_name.replace(' ', '_')}")
            target_person = URIRef(f"{anno_named_graph_uri}/persons/{target_name.replace(' ', '_')}")
            # create father_of as crm:P152_has_parent relation
            anno_named_graph.add((target_person, CRM.P152_has_parent, source_person))
            
            # we could add a relation linked to the concrete annotations in the source
            #source_id = row['source_id']
            #target_id = row['target_id']
            #target_anno = URIRef(f"https://recogito.pelagios.org/annotation/{target_id}")
            #source_anno = URIRef(f"https://recogito.pelagios.org/annotation/{source_id}")
            
        else:
            print(f"don't know what to do with {relation_type=}")

In [ ]:
store.close()